# Title
[]()

# Original scripts from `chat_functions.py`

In [ ]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(documents, site_key, vector_dict=vector_dict, text_splitter=None):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    embeddings_dict[site_key] = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


# Iteration 1

In [6]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(documents, site_key, vector_dict=vector_dict, text_splitter=None):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    embeddings_dict[site_key] = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


def create_retriever_and_description_dicts(params_dict, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], parms_dict[doc_id]['site_key'], 
            vector_dict=vector_dict, 
            text_splitter=parms_dict[doc_id].get('text_splitter', None)
            )
        description_dict[parms_dict[doc_id]['site_key']] = parms_dict['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    1: {
        'site_key': 'recycle',
        'doc_description': recylebc,
        'text_splitter': None
    },
    2: {
        'site_key': 'matttress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}

print(params_dict[1].get('text_splitter', 'hello'))

hello


# *End of Page*